Since this project uses a pretrained doc2vec model with for old version of gensim therefore it only supports `python 2` 

In [93]:
import pandas as pd
import seaborn as sns
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
import gensim.models as g
from gensim.models.doc2vec import  TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_non_alphanum
import nltk
from tqdm import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize
tqdm.pandas()

[nltk_data] Downloading package punkt to /home/dhiab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# ***Reading the Dataset***

In [29]:
data = pd.read_csv("./data/IMDB_Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
%%time
data['sentiment'] = data['sentiment'].progress_apply(lambda x: 1 if x=='positive' else 0)

100%|██████████| 50000/50000 [00:00<00:00, 1240999.12it/s]

CPU times: user 41 ms, sys: 3.95 ms, total: 44.9 ms
Wall time: 44.5 ms


# ***Importing Pretrained Doc2Vec Model and getting the Embeddings of the reviews***

the Doc2vec used in this project was pretrained using a Wikipedia dump in english (millions of articles)

In [31]:
model="./data/doc2vec.bin"



#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

#load model
m = g.Doc2Vec.load(model)

In [32]:
def tok_review(row):
    review=row['review']
    review=stem_text(remove_stopwords(strip_non_alphanum(review)))
    tokenized_review=word_tokenize(review.lower())
    return(tokenized_review)

In [33]:
data_new=data.copy()

In [37]:
data_new['tokenized review']=data_new.progress_apply(tok_review,axis=1)

100%|██████████| 50000/50000 [00:57<00:00, 875.86it/s]


In [38]:
def embed_review(row):
    return(m.infer_vector(row['tokenized review'], alpha=start_alpha, steps=infer_epoch))

In [40]:
data_new['embedded review']=data_new.progress_apply(embed_review,axis=1)

100%|██████████| 50000/50000 [1:46:34<00:00,  7.82it/s]


In [41]:
data_new.head()

,review,sentiment,tokenized review,embedded review
0,One of the other reviewers has mentioned that ...,1,"[on, review, mention, watch, 1, oz, episod, ll...","[0.037001394, 1.3362205, -1.0229214, -0.686279..."
1,A wonderful little production. <br /><br />The...,1,"[a, wonder, littl, product, br, br, the, film,...","[-0.0803674, 0.25143552, -0.27523, -0.3200736,..."
2,I thought this was a wonderful way to spend ti...,1,"[i, thought, wonder, wai, spend, time, hot, su...","[0.63847303, 0.18252392, 0.28718272, 1.1484635..."
3,Basically there's a family where a little boy ...,0,"[basic, s, famili, littl, boi, jake, think, s,...","[-0.08427986, 0.15279457, 0.17338341, 0.311363..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[petter, mattei, s, love, time, monei, visual,...","[0.8196315, -0.08279816, 0.81621045, -0.118137..."


In [42]:
data_new.to_csv('./data/doc2vec_dataset.csv',index=False,sep='\t')